In [ ]:
!pip install lifelines -q --no-index --find-links=cibmtr2024-import/lifelines
!pip install scikit-learn==1.4.0 -q --no-index --find-links=cibmtr2024-import/scikit_learn
!pip install rtdl_num_embeddings -q --no-index --find-links=cibmtr2024-import/rtdl_num_embeddings
!pip install delu -q --no-index --find-links=cibmtr2024-import/delu

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement scikit-learn==1.4.0 (from versions: none)
ERROR: No matching distribution found for scikit-learn==1.4.0


In [2]:
from tabm_reference import Model, make_parameter_groups

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import rtdl_num_embeddings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import average_precision_score

import os
import gc
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import joblib
from torch.utils.data import TensorDataset, DataLoader, Dataset, ConcatDataset

import delu
import math

from collections import OrderedDict
from tqdm import tqdm

In [ ]:
MAIN_PATH = 'avito-train-combined/data/avito-merged-dataset/'
ECOM_PRETRAIN = 'avito-train-combined/data/clip-marqofashionsiglip-marqoecom-top2kaggle/'
RESNET_PATH = 'avito-train-combined/data/resnet-cossim/'
KAGGLE_TOP5 = 'avito-train-combined/data/top5-kaggle/'
BERTA_PATH = 'avito-train-combined/data/berta-pretrained-cossims/'
RUBERT_TINY_OOF_PATH = 'avito-train-combined/data/rubert-folds/'
RUBERT_TINY_PREDS_PATH = 'avito-train-combined/data/rubert-test-preds/'
E5LARGE_OOF_PATH = 'avito-train-combined/data/avito-e5-large-pretrain/'
E5LARGE_PREDS_PATH = 'avito-train-combined/data/avito-e5-large-test/'
REV_RUBERT_TINY_OOF_PATH = 'avito-train-combined/data/name_desc_bert_oof_rev/'
REV_RUBERT_TINY_PREDS_PATH = 'avito-train-combined/data/name-desc-bert-preds-rev/'
USERBGE_COSSIMS_PATH = 'avito-train-combined/data/userbge-cossims/'
RUBERT_BASE_TEST_PREDS_PATH = 'avito-train-combined/data/rubert-fixed-test-preds/'
RUBERT_BASE_TEST_PREDS_REV_PATH = 'avito-train-combined/data/rubert-fixed-test-preds-rev/'
FT_PREDS_PATH = 'avito-train-combined/data/ft-preds2/'
RUBERT_BASE_OOF_PATH = 'avito-train-combined/data/trained-rubert-base-preds/'
RUBERT_BASE_OOF_REV_PATH = 'avito-train-combined/data/trained-rubert-base-preds-rev/'
ROUGE_PATH = 'rouge-avito/'

USE_MEAN_BASE_AND_REV = False

In [4]:
train = pd.read_parquet(MAIN_PATH + 'train_df.parquet')
test = pd.read_parquet(MAIN_PATH + 'test_df.parquet')

In [5]:
to_drop = [
    'category_level_1_1', 'category_level_1_2',
    'category_level_2_1', 'category_level_2_2',
    'category_level_3_1', 'category_level_3_2',
    'category_level_4_1', 'category_level_4_2',
]

train[to_drop] = train[to_drop].fillna('none')

train['unique_cat_1'] = train['category_level_1_1'] + '_' + train['category_level_1_2']
train['unique_cat_2'] = train['category_level_2_1'] + '_' + train['category_level_2_2']
train['unique_cat_3'] = train['category_level_3_1'] + '_' + train['category_level_3_2']
train['unique_cat_4'] = train['category_level_4_1'] + '_' + train['category_level_4_2']

test[to_drop] = test[to_drop].fillna('none')

test['unique_cat_1'] = test['category_level_1_1'] + '_' + test['category_level_1_2']
test['unique_cat_2'] = test['category_level_2_1'] + '_' + test['category_level_2_2']
test['unique_cat_3'] = test['category_level_3_1'] + '_' + test['category_level_3_2']
test['unique_cat_4'] = test['category_level_4_1'] + '_' + test['category_level_4_2']

train.drop(columns=to_drop, axis=1, inplace=True)
gc.collect()

test.drop(columns=to_drop, axis=1, inplace=True)
gc.collect()

0

In [6]:
numerical_features = train.select_dtypes(
    include=['float16', 'float32', 'float64', 'int8', 'int16', 'int32', 'int64']
).columns.to_list()

cols_with_inf = train[numerical_features].columns[np.isinf(train[numerical_features]).any()].tolist()

print("Колонки с inf:", cols_with_inf)

train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

Колонки с inf: ['name_tanimoto', 'name_norm_tanimoto', 'name_en_tanimoto', 'name_mix_tanimoto', 'description_en_tanimoto', 'description_mix_tanimoto', 'name_tokens_w_digits_tanimoto', 'description_tokens_w_digits_tanimoto']


In [7]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                    gc.collect()
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                    gc.collect()
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                    gc.collect()
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
                    gc.collect()
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                    gc.collect()
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                    gc.collect()
                else:
                    df[col] = df[col].astype(np.float64)
                    gc.collect()
        else:
            if df[col].nunique() == 2:
                df[col] = df[col].astype('bool')
            gc.collect()

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    gc.collect()
    
    return df

In [8]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 6521.05 MB
Memory usage after optimization is: 1756.63 MB
Decreased by 73.1%


In [9]:
test = reduce_mem_usage(test)

Memory usage of dataframe is 1728.06 MB
Memory usage after optimization is: 467.78 MB
Decreased by 72.9%


In [10]:
train = train.sort_values(by=['variantid_1', 'variantid_2'])
test = test.sort_values(by=['variantid_1', 'variantid_2'])

In [11]:
# --- IMG FEATURES ---

# pretrain clip
train_clip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_train_CLIP.parquet')
test_clip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_test_CLIP.parquet')

train_clip = train_clip.sort_values(by=['variantid_1', 'variantid_2'])
test_clip = test_clip.sort_values(by=['variantid_1', 'variantid_2'])
train['clip_cosine_sim'] = train_clip['cosine_sim']
test['clip_cosine_sim'] = test_clip['cosine_sim']
del train_clip, test_clip

# pretrain fashion siglip
train_fashionsiglip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_fashion_clip_train.parquet')
test_fashionsiglip = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_fashion_clip_test.parquet')

train_fashionsiglip = train_fashionsiglip.sort_values(by=['variantid_1', 'variantid_2'])
test_fashionsiglip = test_fashionsiglip.sort_values(by=['variantid_1', 'variantid_2'])
train['fashionsiglip_cosine_sim'] = train_fashionsiglip['cosine_sim']
test['fashionsiglip_cosine_sim'] = test_fashionsiglip['cosine_sim']
del train_fashionsiglip, test_fashionsiglip

# pretrain marqo ecom
train_ecom = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_ecomm_train.parquet')
test_ecom = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_embeddings_ecomm_test.parquet')

train_ecom = train_ecom.sort_values(by=['variantid_1', 'variantid_2'])
test_ecom = test_ecom.sort_values(by=['variantid_1', 'variantid_2'])
train['ecom_cosine_sim'] = train_ecom['cosine_sim']
test['ecom_cosine_sim'] = test_ecom['cosine_sim']
del train_ecom, test_ecom

# kaggle top2 model, bugged
# train_top2kaggle = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_top2_kaggle_train.parquet')
# test_top2kaggle = pd.read_parquet(ECOM_PRETRAIN + 'cossim_final_top2_kaggle_test.parquet')

# train_top2kaggle = train_top2kaggle.sort_values(by=['variantid_1', 'variantid_2'])
# test_top2kaggle = test_top2kaggle.sort_values(by=['variantid_1', 'variantid_2'])
# train['top2kaggle_cosine_sim'] = train_top2kaggle['cosine_sim']
# test['top2kaggle_cosine_sim'] = test_top2kaggle['cosine_sim']
# del train_top2kaggle, test_top2kaggle

# kaggle top5 model
train_top5kaggle = pd.read_parquet(KAGGLE_TOP5 + 'cossim_final_concat_train.parquet')
test_top5kaggle = pd.read_parquet(KAGGLE_TOP5 + 'cossim_final_concat_test.parquet')

train_top5kaggle = train_top5kaggle.sort_values(by=['variantid_1', 'variantid_2'])
test_top5kaggle = test_top5kaggle.sort_values(by=['variantid_1', 'variantid_2'])
train['top5kaggle_cosine_sim'] = train_top5kaggle['cosine_sim']
test['top5kaggle_cosine_sim'] = test_top5kaggle['cosine_sim']
del train_top5kaggle, test_top5kaggle

# trained resnet
train_resnet = pd.read_parquet(RESNET_PATH + 'train_resnet_cossim.parquet')
test_resnet = pd.read_parquet(RESNET_PATH + 'test_resnet_cossim.parquet')

train_resnet = train_resnet.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test_resnet = test_resnet.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
train['resnet_cosine_sim'] = train_resnet['cossims_resnet']
test['resnet_cosine_sim'] = test_resnet['cossims_resnet']
del train_resnet, test_resnet

gc.collect()

# --- TEXT FEATURES ---

# pretrain berta
berta_cossims_train_part1 = pd.read_parquet(BERTA_PATH + 'berta_cossims_train_part1.parquet')
berta_cossims_train_part2 = pd.read_parquet(BERTA_PATH + 'berta_cossims_train_part2.parquet')
berta_cossims_train = pd.concat([berta_cossims_train_part1, berta_cossims_train_part2])
berta_cossims_test = pd.read_parquet(BERTA_PATH + 'berta_cossims_test.parquet')

berta_cossims_train = berta_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])
berta_cossims_test = berta_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])
train['berta_cossim'] = berta_cossims_train['berta_cossim']
test['berta_cossim'] = berta_cossims_test['berta_cossim']

del berta_cossims_train_part1, berta_cossims_train_part2, berta_cossims_train, berta_cossims_test
gc.collect()

# trained rubert (cherez zhopu rukamu obuchen)
rubert_oof_fold0 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold0.parquet')
rubert_oof_fold1 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold1.parquet')
rubert_oof_fold2 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold2.parquet')
rubert_oof_fold3 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold3.parquet')
rubert_oof_fold4 = pd.read_parquet(RUBERT_TINY_OOF_PATH + 'name_desc_bert_fold4.parquet')

rubert_test_pred_fold0 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_0.parquet')
rubert_test_pred_fold1 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_1.parquet')
rubert_test_pred_fold2 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_2.parquet')
rubert_test_pred_fold3 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_3.parquet')
rubert_test_pred_fold4 = pd.read_parquet(RUBERT_TINY_PREDS_PATH + 'name_desc_rubert_tiny_turbo_2048_wce_4.parquet')

rubert_oof_fold0.rename(columns={'name_desc_bert_oof1': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold1.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold2.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold3.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)
rubert_oof_fold4.rename(columns={'name_desc_bert_oof4': 'name_desc_rubert_tiny_turbo_2048_wce'}, inplace=True)

rubert_oof_fold0 = rubert_oof_fold0.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold1 = rubert_oof_fold1.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold2 = rubert_oof_fold2.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold3 = rubert_oof_fold3.sort_values(by=['variantid_1', 'variantid_2'])
rubert_oof_fold4 = rubert_oof_fold4.sort_values(by=['variantid_1', 'variantid_2'])

rubert_oof = rubert_oof_fold0['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold1['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold2['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold3['name_desc_rubert_tiny_turbo_2048_wce'] + rubert_oof_fold4['name_desc_rubert_tiny_turbo_2048_wce']

rubert_test_pred_fold0 = rubert_test_pred_fold0.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold1 = rubert_test_pred_fold1.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold2 = rubert_test_pred_fold2.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold3 = rubert_test_pred_fold3.sort_values(by=['variantid_1', 'variantid_2'])
rubert_test_pred_fold4 = rubert_test_pred_fold4.sort_values(by=['variantid_1', 'variantid_2'])

rubert_preds = (rubert_test_pred_fold0['name_desc_rubert_tiny_turbo_2048_wce_0'] + rubert_test_pred_fold1['name_desc_rubert_tiny_turbo_2048_wce_1'] + rubert_test_pred_fold2['name_desc_rubert_tiny_turbo_2048_wce_2'] + rubert_test_pred_fold3['name_desc_rubert_tiny_turbo_2048_wce_3'] + rubert_test_pred_fold4['name_desc_rubert_tiny_turbo_2048_wce_4']) / 5

train['name_desc_rubert_tiny_turbo_2048_wce'] = rubert_oof
test['name_desc_rubert_tiny_turbo_2048_wce'] = rubert_preds

del rubert_oof_fold0, rubert_oof_fold1, rubert_oof_fold2, rubert_oof_fold3, rubert_oof_fold4
del rubert_oof
del rubert_test_pred_fold0, rubert_test_pred_fold1, rubert_test_pred_fold2, rubert_test_pred_fold3, rubert_test_pred_fold4
del rubert_preds
gc.collect()

# pretrain e5large
e5large_cossims_train_part1 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold0.parquet')
e5large_cossims_train_part2 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold1.parquet')
e5large_cossims_train_part3 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold2.parquet')
e5large_cossims_train_part4 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold3.parquet')
e5large_cossims_train_part5 = pd.read_parquet(E5LARGE_OOF_PATH + 'e5large_cossims_fold4.parquet')

e5large_cossims_train = pd.concat([
    e5large_cossims_train_part1,
    e5large_cossims_train_part2,
    e5large_cossims_train_part3,
    e5large_cossims_train_part4,
    e5large_cossims_train_part5
])
e5large_cossims_train = e5large_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])

del e5large_cossims_train_part1, e5large_cossims_train_part2, e5large_cossims_train_part3, e5large_cossims_train_part4, e5large_cossims_train_part5
gc.collect()

e5large_cossims_test_part1 = pd.read_parquet(E5LARGE_PREDS_PATH + 'e5large_cossims_part1.parquet')
e5large_cossims_test_part2 = pd.read_parquet(E5LARGE_PREDS_PATH + 'e5large_cossims_part2.parquet')

e5large_cossims_test = pd.concat([
    e5large_cossims_test_part1,
    e5large_cossims_test_part2,
])
e5large_cossims_test = e5large_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])

del e5large_cossims_test_part1, e5large_cossims_test_part2
gc.collect()

train['e5large_cossim'] = e5large_cossims_train['e5large_cossim']
test['e5large_cossim'] = e5large_cossims_test['e5large_cossim']

del e5large_cossims_train, e5large_cossims_test
gc.collect()

# rev trained rubert (cherez zhopu rukamu obuchen)
rubert_oof_rev = pd.read_parquet(REV_RUBERT_TINY_OOF_PATH + 'name_desc_bert_oof_rev.parquet')
rubert_preds_rev = pd.read_parquet(REV_RUBERT_TINY_PREDS_PATH + 'name_desc_bert_preds_rev.parquet')

rubert_oof_rev = rubert_oof_rev.sort_values(by=['variantid_1', 'variantid_2'])
rubert_preds_rev = rubert_preds_rev.sort_values(by=['variantid_1', 'variantid_2'])
train['name_desc_rubert_tiny_turbo_2048_wce_rev'] = rubert_oof_rev['name_desc_bert_oof_rev']
test['name_desc_rubert_tiny_turbo_2048_wce_rev'] = rubert_preds_rev['name_desc_bert_preds_rev']
del rubert_oof_rev, rubert_preds_rev

gc.collect()

# rubert tta
if USE_MEAN_BASE_AND_REV:
    train['name_desc_rubert_tiny_turbo_2048_wce_tta'] = (train['name_desc_rubert_tiny_turbo_2048_wce'] + train['name_desc_rubert_tiny_turbo_2048_wce_rev']) / 2
    test['name_desc_rubert_tiny_turbo_2048_wce_tta'] = (test['name_desc_rubert_tiny_turbo_2048_wce'] + test['name_desc_rubert_tiny_turbo_2048_wce_rev']) / 2
    del train['name_desc_rubert_tiny_turbo_2048_wce'], train['name_desc_rubert_tiny_turbo_2048_wce_rev']
    del test['name_desc_rubert_tiny_turbo_2048_wce'], test['name_desc_rubert_tiny_turbo_2048_wce_rev']
    gc.collect()

# pretrain userbge
userbge_cossims_train = pd.DataFrame()
userbge_cossims_test = pd.DataFrame()

for i in range(1, 16):
    curr_df = pd.read_parquet(USERBGE_COSSIMS_PATH + f'userbge_cossims_train_part{i}.parquet')
    userbge_cossims_train = pd.concat([userbge_cossims_train, curr_df])
    print(f'{userbge_cossims_train.shape=}')
userbge_cossims_train = userbge_cossims_train.sort_values(by=['variantid_1', 'variantid_2'])

for i in range(1, 5):
    curr_df = pd.read_parquet(USERBGE_COSSIMS_PATH + f'/userbge_cossims_test_part{i}.parquet')
    userbge_cossims_test = pd.concat([userbge_cossims_test, curr_df])
    print(f'{userbge_cossims_test.shape=}')
userbge_cossims_test = userbge_cossims_test.sort_values(by=['variantid_1', 'variantid_2'])

del curr_df
gc.collect()

train['userbge_cossim'] = userbge_cossims_train['userbge_cossim']
test['userbge_cossim'] = userbge_cossims_test['userbge_cossim']

del userbge_cossims_train, userbge_cossims_test
gc.collect()

# --- тут именно до сэмпла! ---

# trained rubert-base test preds
rubert_base_pred = pd.read_csv(
    RUBERT_BASE_TEST_PREDS_PATH + 'rubert_ZAEBAL_SUKA.csv'
).rename(columns={'base_id': 'variantid_1', 'cand_id': 'variantid_2'})
rubert_base_pred = rubert_base_pred.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test['rubert_base_trained'] = rubert_base_pred['probability']

rubert_base_pred_rev = pd.read_csv(
    RUBERT_BASE_TEST_PREDS_REV_PATH + 'rubert_ZAEBAL_SUKA_REV.csv'
).rename(columns={'base_id': 'variantid_1', 'cand_id': 'variantid_2'})
rubert_base_pred_rev = rubert_base_pred_rev.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test['rubert_base_trained_rev'] = rubert_base_pred_rev['probability']

del rubert_base_pred, rubert_base_pred_rev
gc.collect()

# rubert base test tta
if USE_MEAN_BASE_AND_REV:
    test['rubert_base_trained_tta'] = (test['rubert_base_trained'] + test['rubert_base_trained_rev']) / 2
    del test['rubert_base_trained'],  test['rubert_base_trained_rev']
    gc.collect()

# --- sample!!! ---
train = train.sample(len(train), random_state=42)

# trained ft (именно после сэмпла! тут уже все карты в нужном порядке разложены)
fasttext_train = joblib.load(FT_PREDS_PATH + 'oof_preds.pkl')
fasttext_test = joblib.load(FT_PREDS_PATH + 'test_preds.pkl')
fasttext_train_rev = joblib.load(FT_PREDS_PATH + 'oof_preds_rev.pkl')
fasttext_test_rev = joblib.load(FT_PREDS_PATH + 'test_preds_rev.pkl')

train['fasttext'] = fasttext_train
test['fasttext'] = fasttext_test
train['fasttext_rev'] = fasttext_train_rev
test['fasttext_rev'] = fasttext_test_rev

del fasttext_train, fasttext_test, fasttext_train_rev, fasttext_test_rev
gc.collect()

# fasttext tta
if USE_MEAN_BASE_AND_REV:
    train['fasttext_tta'] = (train['fasttext'] + train['fasttext_rev']) / 2
    test['fasttext_tta'] = (test['fasttext'] + test['fasttext_rev']) / 2
    del train['fasttext'], train['fasttext_rev']
    del test['fasttext'], test['fasttext_rev']
    gc.collect()

# trained rubert-base train (именно после сэмпла! тут уже все карты в нужном порядке разложены)
rubert_base_oof = joblib.load(RUBERT_BASE_OOF_PATH + 'oof_preds_rubert_base.joblib')
train['rubert_base_trained'] = rubert_base_oof

rubert_base_oof_rev = joblib.load(RUBERT_BASE_OOF_REV_PATH + 'oof_preds_rubert_base_rev.joblib')
train['rubert_base_trained_rev'] = rubert_base_oof_rev

del rubert_base_oof, rubert_base_oof_rev
gc.collect()

# rubert train test tta
if USE_MEAN_BASE_AND_REV:
    train['rubert_base_trained_tta'] = (train['rubert_base_trained'] + train['rubert_base_trained_rev']) / 2
    del train['rubert_base_trained'],  train['rubert_base_trained_rev']
    gc.collect()

# add rouge features (w/ tta)

train_rouge = pd.read_csv(ROUGE_PATH + 'train_rouge.csv')
test_rouge = pd.read_csv(ROUGE_PATH + 'test_rouge.csv')

train_rouge = train_rouge.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)
test_rouge = test_rouge.sort_values(by=['variantid_1', 'variantid_2']).reset_index(drop=True)

train['rouge_1'] = train_rouge['rouge_1']
train['rouge_2'] = train_rouge['rouge_2']
train['rouge_3'] = train_rouge['rouge_3']
train['rouge_4'] = train_rouge['rouge_4']
train['rouge_s4'] = train_rouge['rouge_s4']
train['rouge_su4'] = train_rouge['rouge_su4']

test['rouge_1'] = test_rouge['rouge_1']
test['rouge_2'] = test_rouge['rouge_2']
test['rouge_3'] = test_rouge['rouge_3']
test['rouge_4'] = test_rouge['rouge_4']
test['rouge_s4'] = test_rouge['rouge_s4']
test['rouge_su4'] = test_rouge['rouge_su4']

userbge_cossims_train.shape=(125303, 3)
userbge_cossims_train.shape=(250606, 3)
userbge_cossims_train.shape=(375909, 3)
userbge_cossims_train.shape=(501212, 3)
userbge_cossims_train.shape=(626515, 3)
userbge_cossims_train.shape=(751818, 3)
userbge_cossims_train.shape=(877121, 3)
userbge_cossims_train.shape=(1002424, 3)
userbge_cossims_train.shape=(1127727, 3)
userbge_cossims_train.shape=(1253030, 3)
userbge_cossims_train.shape=(1378333, 3)
userbge_cossims_train.shape=(1503636, 3)
userbge_cossims_train.shape=(1628939, 3)
userbge_cossims_train.shape=(1754242, 3)
userbge_cossims_train.shape=(1879555, 3)
userbge_cossims_test.shape=(125000, 3)
userbge_cossims_test.shape=(250000, 3)
userbge_cossims_test.shape=(375000, 3)
userbge_cossims_test.shape=(500000, 3)


In [12]:
train.drop(columns=['base_title_image', 'cand_title_image'], axis=1, inplace=True)
test.drop(columns=['base_title_image', 'cand_title_image'], axis=1, inplace=True)
train.drop(columns=['action_date'], inplace=True)

In [13]:
cat_features = [
    'is_same_location',
    'is_same_region',
    'category_level_1_match',
    'category_level_2_match',
    'category_level_3_match',
    'category_level_4_match',
    'category_level_3_fillness',
    'category_level_4_fillness',
    'n_images_fillness',
    'unique_cat_1',
    'unique_cat_2',
    'unique_cat_3',
    'unique_cat_4',
]

train[cat_features] = train[cat_features].astype(str)
test[cat_features] = test[cat_features].astype(str)

In [14]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [15]:
features = [col for col in train.columns if col not in ['group_id', 'is_double', 'variantid_1', 'variantid_2']]
target = 'is_double'

In [16]:
len(features)

470

In [17]:
num_features = [c for c in features if not c in cat_features]

In [18]:
for col in cat_features:
    fill_value = 'missing'
    train[col] = train[col].fillna(fill_value)
    test[col] = test[col].fillna(fill_value)

for col in num_features:
    train[col] = pd.to_numeric(train[col], errors='coerce')
    test[col] = pd.to_numeric(test[col], errors='coerce')

for col in num_features:
    fill_value = train[col].median()
    train[col] = train[col].fillna(fill_value)
    test[col] = test[col].fillna(fill_value)

In [19]:
combined = pd.concat([train, test], axis=0, ignore_index=True)

for c in features:
    if c in cat_features:
        print(f"{c}, ", end="")
        combined[c], _ = combined[c].factorize()
        combined[c] -= combined[c].min()
        combined[c] = combined[c].astype("int32")
        combined[c] = combined[c].astype("category")
    else:
        if combined[c].dtype == "float64":
            combined[c] = combined[c].astype("float32")
        if combined[c].dtype == "int64":
            combined[c] = combined[c].astype("int32")
    
cat_unique = combined[cat_features].nunique().to_list()

train = combined.iloc[:len(train)].reset_index(drop=True).copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

is_same_location, is_same_region, category_level_1_match, category_level_2_match, category_level_3_match, category_level_4_match, category_level_3_fillness, category_level_4_fillness, n_images_fillness, unique_cat_1, unique_cat_2, unique_cat_3, unique_cat_4, 

In [ ]:
folds = 5
train['kfold'] = -1  

target = 'is_double'
sgkf = StratifiedGroupKFold(n_splits=folds)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train[target], groups=train['group_id'])):
    train.loc[val_idx, 'kfold'] = fold

oof_metric = train[['variantid_1', 'variantid_2', 'kfold', 'group_id', 'is_double']].copy()
oof_metric['prediction'] = 0.0

oof_tabm = np.zeros(train.shape[0])
test_tabm = np.zeros((folds, test.shape[0]))

In [21]:
dfsum = pd.DataFrame(train.isna().sum())

In [ ]:
cats_index = [train[features].columns.get_loc(cat) for cat in cat_features]

scaler = StandardScaler()
train[num_features] = scaler.fit_transform(train[num_features])
test[num_features] = scaler.transform(test[num_features])

In [ ]:
X_num = train[num_features].values
X_cat = train[cat_features].values

X_num_test = test[num_features].values
X_cat_test = test[cat_features].values

y = train[target].values

test_dl = DataLoader(
    TensorDataset(
        torch.tensor(X_num_test, dtype=torch.float32),
        torch.tensor(X_cat_test, dtype=torch.int64)
    ), 
    batch_size=1024, 
    shuffle=False
)

In [ ]:
n_cont_features = len(num_features)
n_cat_features = len(cat_features)
n_classes = 2
cat_cardinalities = cat_unique

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
arch_type = 'tabm-mini'

class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)

In [ ]:
TRAIN = True
MODEL_SAVE_PATH = "tabm_models_wce"

val_wce_scores = []
val_prauc_scores = []    

if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

for i, (train_index, val_index) in enumerate(sgkf.split(train[features], train[target], groups=train['group_id'])):
    model_path = os.path.join(MODEL_SAVE_PATH, f"model_fold_{i}.pt")

    best = {
        "val": -math.inf,
        "epoch": -1,
    }
    ds_true = oof_metric.loc[oof_metric.kfold == i, ["variantid_1", "variantid_2", "is_double", "group_id"]].copy().reset_index(drop=True)
    ds_pred = oof_metric.loc[oof_metric.kfold == i, ["variantid_1", "variantid_2"]].copy().reset_index(drop=True)

    X_num_train = X_num[train_index]
    X_cat_train = X_cat[train_index]
    y_train = y[train_index]

    X_num_val = X_num[val_index]
    X_cat_val = X_cat[val_index]
    y_val_all = y[val_index]

    train_dl = DataLoader(
        TensorDataset(
            torch.tensor(X_num_train, dtype=torch.float32), 
            torch.tensor(X_cat_train, dtype=torch.int64), 
            torch.tensor(y_train, dtype=torch.float32)
        ), 
        batch_size=32, 
        shuffle=True
    )
    valid_dl = DataLoader(
        TensorDataset(
            torch.tensor(X_num_val, dtype=torch.float32), 
            torch.tensor(X_cat_val, dtype=torch.int64), 
            torch.tensor(y_val_all, dtype=torch.float32)
        ), 
        batch_size=32, 
        shuffle=False
    )

    bins = rtdl_num_embeddings.compute_bins(torch.tensor(X_num_train, dtype=torch.float32))

    model = Model(
        n_num_features=n_cont_features,
        cat_cardinalities=cat_cardinalities,
        n_classes=n_classes,
        backbone={
            'type': 'MLP',
            'n_blocks': 3 ,
            'd_block': 512,
            'dropout': 0.1,
        },
        bins=bins,
        num_embeddings=(
            None
            if bins is None
            else {
                'type': 'PiecewiseLinearEmbeddings',
                'd_embedding': 64,
                'activation': True,
                'version': 'B',
            }
        ),
        arch_type=arch_type,
        k=32,
    ).to(device)

    if TRAIN:
        optimizer = torch.optim.AdamW(
            make_parameter_groups(model),
            lr=1e-4,
            weight_decay=1e-3 ,
        )

        patience = 15
        early_stopping = delu.tools.EarlyStopping(patience, mode="max")

        for epoch in range(100):
            model.train()   
            with tqdm(train_dl, total=len(train_dl), leave=True) as phar:
                for train_tensor in phar:
                    optimizer.zero_grad()
                    X_num_train, X_cat_train, y_train = [t.to(device) for t in train_tensor]

                    output = model(X_num_train, X_cat_train).squeeze(-1)
                    loss = loss_fn(output.flatten(0, 1), y_train.repeat_interleave(32))
                    loss.backward()
                    optimizer.step()

                    phar.set_postfix(OrderedDict(epoch=f'{epoch+1}/{100}', loss=f'{loss.item():.6f}'))
                    phar.update(1)

            model.eval()
            valid_pred_list = []
            for valid_tensor in valid_dl:
                X_num_val, X_cat_val, y_val = [t.to(device) for t in valid_tensor]
                with torch.no_grad():
                    output = model(X_num_val, X_cat_val).squeeze(-1)
                valid_pred_list.append((output.mean(1).cpu().numpy(), y_val.cpu().numpy()))

            valid_pred = np.concatenate([p[0] for p in valid_pred_list])
            valid_true = np.concatenate([p[1] for p in valid_pred_list])
            val_prauc = average_precision_score(valid_true, valid_pred)
            
            ds_pred["prediction"] = valid_pred
            
            if val_prauc > best["val"]:
                print(f'{val_prauc=}')
                best = {"val": val_prauc, "epoch": epoch, "pred": valid_pred}
                torch.save(model.state_dict(), model_path)

            early_stopping.update(val_prauc)
            if early_stopping.should_stop():
                print("early stopping")
                break
    else:  
        mp = os.path.join('/kaggle/working/', mp)
        print(f"loading model from {mp}")
        model.load_state_dict(torch.load(mp))

    oof_tabm[val_index] = best['pred']
    val_prauc_scores.append(best['val'])

    ds_pred["prediction"] = best['pred']
    
    model.load_state_dict(torch.load(model_path))
    model.eval()
    test_pred_list = []
    with torch.no_grad():
        for test_tensor in test_dl:
            X_num_test, X_cat_test = [t.to(device) for t in test_tensor]
            output = model(X_num_test, X_cat_test).squeeze(-1)
            test_pred_list.append(output.mean(1).cpu().numpy())

    test_pred = np.concatenate([p for p in test_pred_list])
    test_tabm[i] = test_pred

    print(" *************************************************************************************** ")
    print(f"fold {i+1} prauc: {val_prauc:.6f} - это просто ахуенно!")
    print("\n")
    print(" *************************************************************************************** ")

In [ ]:
print("Mean Validation PRAUC: {:.6f}".format(np.mean(val_prauc_scores)))
print("OOF PRAUC: {:.6f}".format(average_precision_score(train[target], oof_tabm)))

In [ ]:
test_mean = np.mean(test_tabm, axis=0)
submission = test[["variantid_1", "variantid_2"]].copy()
submission["prediction"] = test_mean
submission.to_csv("tabm_submission.csv", index=False)